In [1]:
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Football-football-Vs-Rugby-rugby_football-Image-Classification'

In [2]:
def load_data():
    labels = {}
    idx = -1
    labels_r = {}
    data = []
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))/255.0
            data.append([img,idx])
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.25)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.99s/it]


In [4]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [6]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [7]:
def predict(model):
    model.eval()
    for file in os.listdir('./test_data/'):
        img = cv2.imread(f'./test_data/{file}')
        img = cv2.resize(img,(112,112))/255.0
        preds = model(torch.from_numpy(np.array(img)).view(-1,3,112,112).to(device).float())
        preds = int(torch.round(preds))
        plt.figure(figsize=(12,6))
        plt.imshow(img)
        plt.title(f'{labels_r[preds]}')
        plt.savefig(f'./preds/{file}')
        plt.close()
    model.train()

In [8]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,6,(5,5))
        self.conv1batchnorm = BatchNorm2d(6)
        self.conv2 = Conv2d(6,9,(5,5))
        self.conv2batchnorm = BatchNorm2d(9)
        self.conv3 = Conv2d(9,12,(5,5))
        self.conv3batchnorm = BatchNorm2d(12)
        self.linear1 = Linear(12*10*10,128)
        self.linear1batchnorm = BatchNorm1d(128)
        self.linear2 = Linear(128,256)
        self.linear2batchnorm = BatchNorm1d(256)
        self.linear3 = Linear(256,128)
        self.linear3batchnorm = BatchNorm1d(128)
        self.output = Linear(128,1)
        self.output_ac = Sigmoid()
    
    def forward(self,X):
        preds = self.max_pool2d(self.activation(self.conv1batchnorm(self.conv1(X))))
        preds = self.max_pool2d(self.activation(self.conv2batchnorm(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3batchnorm(self.conv3(preds))))
#         print(preds.shape)
        preds = preds.view(-1,12*10*10)
        preds = self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds = self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds = self.activation(self.linear3batchnorm(self.linear3(preds)))
        preds = self.output_ac(self.output(preds))
        return preds

In [9]:
model = Model().to(device)

In [10]:
criterion = BCELoss()

In [11]:
from torch.optim import *

In [12]:
optimizer = Adam(model.parameters(),lr=0.001)

In [13]:
epochs = 100

In [14]:
batch_size = 32

In [15]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,112,112).to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
    predict(model)
    for file in os.listdir('./preds/'):
        wandb.log({f'Img/{file}':wandb.Image(cv2.imread(f'./preds/{file}'))})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|                                                   | 0/100 [00:00<?, ?it/s]

torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32

  1%|▍                                          | 1/100 [00:01<02:56,  1.78s/it]

torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32, 12, 10, 10])
torch.Size([32

...

torch.Size([1, 12, 10, 10])


KeyboardInterrupt: 

Error in callback <function _WandbInit._pause_backend at 0x7fdc79b5f9e0> (for post_run_cell):


Process wandb_internal:
Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/indika/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/internal/internal.py", line 152, in wandb_internal
    thread.join()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


KeyboardInterrupt: 